In [1]:
from qdrant_client import QdrantClient
from qdrant_client.models import PointStruct, VectorParams
from sentence_transformers import SentenceTransformer
import numpy as np
from pathlib import Path
import json
import torch
#If you already saved the model locally and are using docker
client = QdrantClient(host="localhost", port=6333, prefer_grpc=True, timeout=1000)
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SentenceTransformer("models/multilingual-e5-large", device=device)

# Run docker container before
# docker run -d --name qdrant -p 6333:6333 -p 6334:6334 -v /home/Loris/EPFL/MA3/ML/project2/project2Rag/qdrant_storage:/qdrant/storage qdrant/qdrant:latest

# VERY IMPORTANT : to query the remote qdrand docker container over ssh, run the following in a terminale before:
# ssh -L 6333:localhost:6333 -L 6334:localhost:6334 <your username>@dclgpusrv.epfl.ch -N
#Make sure to have your ssh keys before
#This forwards the qdrant ports to your local machine
#If you are outside EPFL network, use VPN

/localhome/ltran/.conda/envs/DCL_WIKI_RAG/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import polars as pl
from tqdm import tqdm
import gc

def evaluate_full_dataset(
    client,
    model,
    parquet_path: str = "articles_fr_merged.parquet",
    collection_name: str = "wikipedia_fr",
    batch_size: int = 64,
    min_link_count: int = 5,  # Only hrefs appearing 5+ times (more reliable)
    max_hrefs: int = None,    # None = all hrefs
    checkpoint_every: int = 5000
) -> pl.DataFrame:
    """
    Evaluate on full dataset with checkpointing.
    """
    print("📂 Extracting ALL unique href_decoded...")
    
    # Extract all unique hrefs
    unique_hrefs = (
        pl.scan_parquet(parquet_path)
        .select(['links'])
        .filter(pl.col('links').is_not_null())
        .with_columns(
            pl.col('links').list.eval(
                pl.element().struct.field('href_decoded')
            ).alias('href_list')
        )
        .select(['href_list'])
        .explode('href_list')
        .rename({'href_list': 'href_decoded'})
        .filter(pl.col('href_decoded').is_not_null())
        .filter(pl.col('href_decoded').str.len_chars() > 1)
        .filter(~pl.col('href_decoded').str.starts_with('#'))  # Skip anchor fragments
        .group_by('href_decoded')
        .agg(pl.len().alias('count'))
        .filter(pl.col('count') >= min_link_count)
        .sort('count', descending=True)
        .collect(streaming=True)
    )
    
    if max_hrefs:
        unique_hrefs = unique_hrefs.head(max_hrefs)
    
    total_hrefs = len(unique_hrefs)
    total_links = unique_hrefs['count'].sum()
    
    print(f"✅ Found {total_hrefs:,} unique href_decoded")
    print(f"   Representing {total_links:,} links")
    print(f"   Min occurrences: {min_link_count}")
    
    # Process in batches
    href_list = unique_hrefs['href_decoded'].to_list()
    count_list = unique_hrefs['count'].to_list()
    
    results = []
    exact_count = 0
    mismatch_count = 0
    
    print(f"\n🔮 Processing {total_hrefs:,} href_decoded...")
    
    for i in tqdm(range(0, total_hrefs, batch_size), desc="Evaluating"):
        batch_hrefs = href_list[i:i + batch_size]
        batch_counts = count_list[i:i + batch_size]
        
        # Batch encode
        queries = [f"query: {href}" for href in batch_hrefs]
        embeddings = model.encode(
            queries,
            normalize_embeddings=True,
            convert_to_numpy=True,
            show_progress_bar=False
        )
        
        # Query Qdrant
        for href, count, emb in zip(batch_hrefs, batch_counts, embeddings):
            try:
                res = client.query_points(
                    collection_name=collection_name,
                    query=emb.tolist(),
                    limit=1,
                    score_threshold=0.3
                ).points
                
                if res:
                    pred_title = res[0].payload.get('title', '') or ''
                    similarity = res[0].score
                    is_exact = href.lower().strip() == pred_title.lower().strip()
                else:
                    pred_title = ''
                    similarity = 0.0
                    is_exact = False
                
                results.append({
                    'href_decoded': href,
                    'count': count,
                    'predicted_title': pred_title,
                    'similarity': similarity,
                    'is_exact': is_exact
                })
                
                if is_exact:
                    exact_count += 1
                else:
                    mismatch_count += 1
                    
            except Exception as e:
                results.append({
                    'href_decoded': href,
                    'count': count,
                    'predicted_title': '',
                    'similarity': 0.0,
                    'is_exact': False
                })
                mismatch_count += 1
        
        # Checkpoint progress
        if (i + batch_size) % checkpoint_every == 0:
            processed = min(i + batch_size, total_hrefs)
            current_acc = 100 * exact_count / processed
            print(f"\n   📊 Progress: {processed:,}/{total_hrefs:,} ({100*processed/total_hrefs:.1f}%)")
            print(f"   📈 Running accuracy: {current_acc:.2f}%")
        
        # Memory cleanup
        del embeddings
        if i % (batch_size * 50) == 0:
            gc.collect()
    
    # Create results DataFrame
    results_df = pl.DataFrame(
        results,
        schema={
            'href_decoded': pl.Utf8,
            'count': pl.Int64,
            'predicted_title': pl.Utf8,
            'similarity': pl.Float64,
            'is_exact': pl.Boolean
        }
    )
    
    # Final statistics
    print("\n" + "="*70)
    print("📊 FULL DATASET EVALUATION RESULTS")
    print("="*70)
    
    total_exact = results_df.filter(pl.col('is_exact'))['count'].sum()
    total_mismatch = results_df.filter(~pl.col('is_exact'))['count'].sum()
    
    unweighted_acc = 100 * exact_count / total_hrefs
    weighted_acc = 100 * total_exact / total_links
    
    print(f"\n📈 Unweighted (by unique href):")
    print(f"   Total: {total_hrefs:,}")
    print(f"   ✅ Exact: {exact_count:,} ({unweighted_acc:.2f}%)")
    print(f"   ❌ Mismatch: {mismatch_count:,} ({100-unweighted_acc:.2f}%)")
    
    print(f"\n📈 Weighted (by link frequency):")
    print(f"   Total links: {total_links:,}")
    print(f"   ✅ Exact: {total_exact:,} ({weighted_acc:.2f}%)")
    print(f"   ❌ Mismatch: {total_mismatch:,} ({100-weighted_acc:.2f}%)")
    
    # Similarity stats
    valid_sims = results_df.filter(pl.col('similarity') > 0)['similarity']
    print(f"\n📊 Similarity Distribution:")
    print(f"   Mean: {valid_sims.mean():.4f}")
    print(f"   Median: {valid_sims.median():.4f}")
    print(f"   Std: {valid_sims.std():.4f}")
    
    # By frequency buckets
    print(f"\n📊 Accuracy by Link Frequency:")
    for min_c, max_c, label in [(5, 100, "Rare (5-100)"), (100, 1000, "Medium (100-1K)"), 
                                 (1000, 10000, "Common (1K-10K)"), (10000, None, "Very Common (10K+)")]:
        if max_c:
            bucket = results_df.filter((pl.col('count') >= min_c) & (pl.col('count') < max_c))
        else:
            bucket = results_df.filter(pl.col('count') >= min_c)
        
        if len(bucket) > 0:
            bucket_exact = bucket.filter(pl.col('is_exact')).height
            bucket_acc = 100 * bucket_exact / len(bucket)
            print(f"   {label:20}: {bucket_acc:>5.1f}% ({bucket_exact:,}/{len(bucket):,})")
    
    return results_df


# Run full evaluation (adjust max_hrefs if needed)
full_results = evaluate_full_dataset(
    client,
    model,
    parquet_path="articles_fr_merged.parquet",
    collection_name="wikipedia_fr",
    batch_size=64,
    min_link_count=5,      # Only hrefs appearing 5+ times
    max_hrefs=None,       # Start with 50K, increase to None for full
    checkpoint_every=10000
)

# Save results
full_results.write_parquet("full_href_evaluation_2.parquet")
print(f"\n💾 Results saved to full_href_evaluation_2.parquet")

📂 Extracting ALL unique href_decoded...


/tmp/ipykernel_2373837/1703159120.py:40: DeprecationWarning: the `streaming` parameter was deprecated in 1.25.0; use `engine` instead.
  .collect(streaming=True)


✅ Found 1,078,890 unique href_decoded
   Representing 51,065,598 links
   Min occurrences: 5

🔮 Processing 1,078,890 href_decoded...


Evaluating:   4%|▎         | 625/16858 [06:19<2:37:41,  1.72it/s]


   📊 Progress: 40,000/1,078,890 (3.7%)
   📈 Running accuracy: 37.23%


...


   📊 Progress: 1,000,000/1,078,890 (92.7%)
   📈 Running accuracy: 44.24%


Evaluating:  96%|█████████▋| 16250/16858 [2:31:55<05:38,  1.80it/s]


   📊 Progress: 1,040,000/1,078,890 (96.4%)
   📈 Running accuracy: 44.22%


Evaluating: 100%|██████████| 16858/16858 [2:37:34<00:00,  1.78it/s]



📊 FULL DATASET EVALUATION RESULTS

📈 Unweighted (by unique href):
   Total: 1,078,890
   ✅ Exact: 477,158 (44.23%)
   ❌ Mismatch: 601,732 (55.77%)

📈 Weighted (by link frequency):
   Total links: 51,065,598
   ✅ Exact: 18,541,916 (36.31%)
   ❌ Mismatch: 32,523,682 (63.69%)

📊 Similarity Distribution:
   Mean: 0.8665
   Median: 0.8670
   Std: 0.0221

📊 Accuracy by Link Frequency:
   Rare (5-100)        :  44.6% (450,672/1,010,996)
   Medium (100-1K)     :  39.6% (24,887/62,868)
   Common (1K-10K)     :  32.4% (1,547/4,780)
   Very Common (10K+)  :  21.1% (52/246)

💾 Results saved to full_href_evaluation_2.parquet


In [6]:
import polars as pl
import numpy as np

def display_detailed_results(parquet_path: str = "full_href_evaluation.parquet"):
    """
    Load and display comprehensive results from saved evaluation.
    """
    print("📂 Loading results...")
    df = pl.read_parquet(parquet_path)
    
    total_hrefs = len(df)
    total_links = df['count'].sum()
    exact_df = df.filter(pl.col('is_exact'))
    mismatch_df = df.filter(~pl.col('is_exact'))
    
    # ============================================================
    # 1. OVERALL STATISTICS
    # ============================================================
    print("\n" + "="*80)
    print("📊 1. OVERALL STATISTICS")
    print("="*80)
    
    exact_count = exact_df.height
    mismatch_count = mismatch_df.height
    
    print(f"\n📈 Unweighted (by unique href_decoded):")
    print(f"   Total unique hrefs: {total_hrefs:,}")
    print(f"   ✅ Exact matches:   {exact_count:,} ({100*exact_count/total_hrefs:.2f}%)")
    print(f"   ❌ Mismatches:      {mismatch_count:,} ({100*mismatch_count/total_hrefs:.2f}%)")
    
    exact_links = exact_df['count'].sum()
    mismatch_links = mismatch_df['count'].sum()
    
    print(f"\n📈 Weighted (by link frequency):")
    print(f"   Total links: {total_links:,}")
    print(f"   ✅ Exact matches:   {exact_links:,} ({100*exact_links/total_links:.2f}%)")
    print(f"   ❌ Mismatches:      {mismatch_links:,} ({100*mismatch_links/total_links:.2f}%)")
    
    # ============================================================
    # 2. SIMILARITY DISTRIBUTION
    # ============================================================
    print("\n" + "="*80)
    print("📊 2. SIMILARITY SCORE DISTRIBUTION")
    print("="*80)
    
    valid_df = df.filter(pl.col('similarity') > 0)
    
    print(f"\n📊 Overall (n={len(valid_df):,}):")
    print(f"   Mean:   {valid_df['similarity'].mean():.4f}")
    print(f"   Median: {valid_df['similarity'].median():.4f}")
    print(f"   Std:    {valid_df['similarity'].std():.4f}")
    print(f"   Min:    {valid_df['similarity'].min():.4f}")
    print(f"   Max:    {valid_df['similarity'].max():.4f}")
    
    print(f"\n📊 By Match Type:")
    for label, subset in [("Exact matches", exact_df), ("Mismatches", mismatch_df)]:
        subset_valid = subset.filter(pl.col('similarity') > 0)
        if len(subset_valid) > 0:
            print(f"   {label}:")
            print(f"      Mean: {subset_valid['similarity'].mean():.4f}, Median: {subset_valid['similarity'].median():.4f}")
    
    # Similarity histogram
    print(f"\n📊 Similarity Histogram:")
    for low, high in [(0.7, 0.75), (0.75, 0.8), (0.8, 0.85), (0.85, 0.9), (0.9, 0.95), (0.95, 1.0)]:
        bucket = valid_df.filter((pl.col('similarity') >= low) & (pl.col('similarity') < high))
        exact_in_bucket = bucket.filter(pl.col('is_exact')).height
        total_in_bucket = bucket.height
        if total_in_bucket > 0:
            acc = 100 * exact_in_bucket / total_in_bucket
            bar = "█" * int(total_in_bucket / len(valid_df) * 50)
            print(f"   [{low:.2f}-{high:.2f}): {total_in_bucket:>6,} ({acc:>5.1f}% exact) {bar}")
    
    # ============================================================
    # 3. ACCURACY BY FREQUENCY BUCKET
    # ============================================================
    print("\n" + "="*80)
    print("📊 3. ACCURACY BY LINK FREQUENCY")
    print("="*80)
    
    buckets = [
        (5, 10, "Very Rare (5-10)"),
        (10, 50, "Rare (10-50)"),
        (50, 100, "Uncommon (50-100)"),
        (100, 500, "Medium (100-500)"),
        (500, 1000, "Common (500-1K)"),
        (1000, 5000, "Frequent (1K-5K)"),
        (5000, 10000, "Very Frequent (5K-10K)"),
        (10000, 50000, "High Frequency (10K-50K)"),
        (50000, None, "Extremely High (50K+)")
    ]
    
    print(f"\n{'Frequency Bucket':<25} {'Total':>8} {'Exact':>8} {'Acc %':>8} {'Links':>12}")
    print("-" * 65)
    
    for min_c, max_c, label in buckets:
        if max_c:
            bucket = df.filter((pl.col('count') >= min_c) & (pl.col('count') < max_c))
        else:
            bucket = df.filter(pl.col('count') >= min_c)
        
        if bucket.height > 0:
            bucket_exact = bucket.filter(pl.col('is_exact')).height
            bucket_acc = 100 * bucket_exact / bucket.height
            bucket_links = bucket['count'].sum()
            print(f"{label:<25} {bucket.height:>8,} {bucket_exact:>8,} {bucket_acc:>7.1f}% {bucket_links:>12,}")
    
    # ============================================================
    # 4. TOP EXACT MATCHES
    # ============================================================
    print("\n" + "="*80)
    print("📊 4. TOP EXACT MATCHES (by frequency)")
    print("="*80)
    
    top_exact = exact_df.sort('count', descending=True).head(20)
    print(f"\n{'href_decoded':<40} {'sim':>6} {'count':>10}")
    print("-" * 60)
    for row in top_exact.iter_rows(named=True):
        print(f"{row['href_decoded'][:39]:<40} {row['similarity']:>6.3f} {row['count']:>10,}")
    
    # ============================================================
    # 5. TOP MISMATCHES (Most Impactful Errors)
    # ============================================================
    print("\n" + "="*80)
    print("📊 5. TOP MISMATCHES (by frequency - most impactful errors)")
    print("="*80)
    
    top_mismatch = mismatch_df.sort('count', descending=True).head(20)
    print(f"\n{'href_decoded':<30} {'predicted':<30} {'sim':>5} {'count':>10}")
    print("-" * 80)
    for row in top_mismatch.iter_rows(named=True):
        href = row['href_decoded'][:29]
        pred = row['predicted_title'][:29] if row['predicted_title'] else 'N/A'
        print(f"{href:<30} {pred:<30} {row['similarity']:>5.2f} {row['count']:>10,}")
    
    # ============================================================
    # 6. MISMATCH ANALYSIS (FIXED - no regex)
    # ============================================================
    print("\n" + "="*80)
    print("📊 6. MISMATCH PATTERN ANALYSIS")
    print("="*80)
    
    mismatch_with_pred = mismatch_df.filter(pl.col('predicted_title') != '')
    
    # Count partial matches manually (avoid regex issues)
    partial_count = 0
    partial_examples = []
    
    for row in mismatch_with_pred.head(10000).iter_rows(named=True):  # Sample for speed
        href_lower = row['href_decoded'].lower()
        pred_lower = row['predicted_title'].lower() if row['predicted_title'] else ''
        
        if href_lower in pred_lower or pred_lower in href_lower:
            partial_count += 1
            if len(partial_examples) < 10:
                partial_examples.append(row)
    
    print(f"\n📊 Mismatch Categories (sampled from first 10K):")
    print(f"   Total mismatches: {mismatch_count:,}")
    print(f"   Partial matches (substring): ~{partial_count:,}")
    
    if partial_examples:
        print(f"\n📊 Partial Match Examples:")
        for row in partial_examples:
            print(f"   '{row['href_decoded'][:30]}' → '{row['predicted_title'][:30]}' (n={row['count']:,})")
    
    # ============================================================
    # 7. HIGH SIMILARITY MISMATCHES
    # ============================================================
    print("\n" + "="*80)
    print("📊 7. HIGH SIMILARITY MISMATCHES (sim > 0.88)")
    print("="*80)
    
    high_sim_mismatch = mismatch_df.filter(pl.col('similarity') > 0.88).sort('similarity', descending=True).head(15)
    print(f"\n{'href_decoded':<30} {'predicted':<30} {'sim':>5} {'count':>8}")
    print("-" * 78)
    for row in high_sim_mismatch.iter_rows(named=True):
        href = row['href_decoded'][:29]
        pred = row['predicted_title'][:29] if row['predicted_title'] else 'N/A'
        print(f"{href:<30} {pred:<30} {row['similarity']:>5.3f} {row['count']:>8,}")
    
    # ============================================================
    # 8. LOW SIMILARITY EXACT MATCHES
    # ============================================================
    print("\n" + "="*80)
    print("📊 8. LOW SIMILARITY EXACT MATCHES (sim < 0.85)")
    print("="*80)
    
    low_sim_exact = exact_df.filter(pl.col('similarity') < 0.85).sort('similarity').head(15)
    print(f"\n{'href_decoded':<50} {'sim':>6} {'count':>8}")
    print("-" * 68)
    for row in low_sim_exact.iter_rows(named=True):
        print(f"{row['href_decoded'][:49]:<50} {row['similarity']:>6.3f} {row['count']:>8,}")
    
    # ============================================================
    # 9. SUMMARY
    # ============================================================
    print("\n" + "="*80)
    print("📊 9. SUMMARY")
    print("="*80)
    
    mismatch_sim_mean = mismatch_df.filter(pl.col('similarity') > 0)['similarity'].mean()
    
    print(f"""
    ┌─────────────────────────────────────────────────────────────────┐
    │  SEMANTIC SIMILARITY LINK PREDICTION EVALUATION                 │
    ├─────────────────────────────────────────────────────────────────┤
    │  Total unique href_decoded:  {total_hrefs:>10,}                        │
    │  Total links represented:    {total_links:>10,}                        │
    ├─────────────────────────────────────────────────────────────────┤
    │  UNWEIGHTED ACCURACY:        {100*exact_count/total_hrefs:>10.2f}%                       │
    │  WEIGHTED ACCURACY:          {100*exact_links/total_links:>10.2f}%                       │
    ├─────────────────────────────────────────────────────────────────┤
    │  Mean similarity (exact):    {exact_df['similarity'].mean():>10.4f}                        │
    │  Mean similarity (mismatch): {mismatch_sim_mean:>10.4f}                        │
    └─────────────────────────────────────────────────────────────────┘
    """)
    
    return df


# Load and display results
df = display_detailed_results("full_href_evaluation.parquet")

📂 Loading results...

📊 1. OVERALL STATISTICS

📈 Unweighted (by unique href_decoded):
   Total unique hrefs: 1,078,890
   ✅ Exact matches:   477,158 (44.23%)
   ❌ Mismatches:      601,732 (55.77%)

📈 Weighted (by link frequency):
   Total links: 51,065,598
   ✅ Exact matches:   18,541,916 (36.31%)
   ❌ Mismatches:      32,523,682 (63.69%)

📊 2. SIMILARITY SCORE DISTRIBUTION

📊 Overall (n=1,078,890):
   Mean:   0.8665
   Median: 0.8670
   Std:    0.0221
   Min:    0.7642
   Max:    0.9606

📊 By Match Type:
   Exact matches:
      Mean: 0.8743, Median: 0.8739
   Mismatches:
      Mean: 0.8603, Median: 0.8597

📊 Similarity Histogram:
   [0.75-0.80):    274 (  2.6% exact) 
   [0.80-0.85): 257,450 ( 17.7% exact) ███████████
   [0.85-0.90): 753,056 ( 51.6% exact) ██████████████████████████████████
   [0.90-0.95): 68,094 ( 63.2% exact) ███
   [0.95-1.00):     16 ( 43.8% exact) 

📊 3. ACCURACY BY LINK FREQUENCY

Frequency Bucket             Total    Exact    Acc %        Links
----------------